In [6]:
import sys
import json
import re
import pandas as pd
import pickle

In [63]:
lib_hashtags = set([ht.lower() for ht in [
    'resist',
    'impeachtrump',
    'p2',
    'bluewave2018'
    'fucktrump',
    'resistance',
    'bluewave',
    'fbr', 
    'flipitblue',
    'votebluetosaveamerica',
    'votewithbeto', 
    'Votebeto',
    'betofortexas',
    'gopvotingblue',
    'resisttrump',
    'iamawakenow', 
    'votethemout2018',
    'votebluetoendthisnightmare',
    'trumplies',
    'trumpliesmatter', 
    'goplies',
    'factsmatter',
    'flipthe49th',
    'VoteScholten4Iowa',
    'thisisnotnormal',
    'takeitback',
    'blacklivesmatter',
    'dumptrump',
    'antitrump',
    'protectourcare',
    'votedem',
    'FlipTheHouse',
    'FlipTheSenate',
    'RejectTheNRA',
    'VoteBlue',
    'BlueWave2018',
    'GunSense',
    'flipthe45th',
    'VoteBlueToSaveHealthcare',
    'OpenBorders',
    'VoteBlue2018',
    'CABlueWAVE',
    'VoteBlueThisTuesday',
    'VoteBLUENoMatterWho',
    'Beto4Texas',
    'VoteBlueToSaveAmerica',
    'VoterSuppressionTactics',
    'AbolishIce',
    'Bernie2020',
    'VoterSuppression',
    'VoteForTimCanova',
    'VoteBlueToSaveAmericanDemocracy',
    'VoteJanz',
    'VoteClaire',
    'VoteforAndrewJanz',
    'ReplaceNunes',
    'RepublicanLiars',
    'ResistandWin',
    'medicareforall',
    'freecollege',
    'ObamaLegacy',
    'DemsSoDiverse',
    'FLIPITVOTEBLUE',
    'PreExistingConditions',
    'goodtrouble',
    'REPUBLICUNT',
    'BoycottTysonFoods',
    'StaceyAbrams4GAgov',
    'GunSenseCandidate',
    'TeamAbrams',
    'VoteBlueToSaveAmerica',
    'medicare4all',
    'neonazi',
    'neonazis',
    'whitesupremacist',
    'whitesupremacists',
    'turngablue',
    'prochoice',
    'VoteScholten4Iowa',
    'VoteWalkerOut',
    'LGBT',
    'lgbtq',
    'lgbtq+',
    'GOPTaxScam',
    'LGBTQIA',
    'VoteBlueToSaveDemocracy',
    'VoteGOPOut',
    'StandIndivisible',
    'FlipThe8th',
    'waveCast',
    'WaveCastWA',
    'BuildTheBlueWave',
    '14yearsisenough',
    'flipthe3rd',
    'flipca25',
    'VoteBetoORourke4Texas',
    'ResistanceRises',
    'feelthebern',
]])
cons_hashtags = set([ht.lower() for ht in [
    'trumptrain',
    'maga',
    'walkaway',
    'qanon',
    'changethemedia',
    'greatawakening',
    'americafirst',
    'wwg1wga',
    'tcot',
    'kag',
    'patriotsunited',
    'qarmy',
    'ccot',
    'thegreatawakening',
    'q',
    'trump2020',
    'makeamericagreatagain',
    'liberalismisamentaldisorder',
    'trumpsarmy',
    'voteredtosaveamerica',
    'votered',
    'unitedvotered',
    'supportice',
    'keepredstatesred',
    'turnbluestatesred',
    'voteredtosavecalifornia',
    'MadMaxine',
    'VoteRedToSaveAmerica2018',
    'ResistTheResistance',
    'VoteAntonioToSaveCalifornia',
    'VoteRed2018',
    'GetMaxineOut',
    'RedWave2018',
    'blexit',
    'PledgeToVoteRed',
    'MAGA2020',
    'VoteRedSaveAmerica2018',
    'RedWave2018andBeyond',
    'redcalifornia',
    'VoteDemsOut',
    'MakeAmericsGreatAgain',
    'REDWAVECOMING',
    'RedTsunami',
    'VoteAntonioSabato',
    'VoteRedOrAmericaIsDead',
    'RedWave',
    'VoteRedSaveAmerica',
    'VoteRedMidterms2018',
    'CaravanInvasion',
    'WalkAwayFromDemocrats',
    'RedNovemeber',
    'RedNovember',
    'KeepFloridaRed',
    'BuildTheWall',
    'DemocratsHateAmerica',
    'VoteAntonio2018',
    'IVotedRED',
    'DefundPlannedParenthood',
    'MSM',
    'VoteRedToSaveCalifornia',
    'LyingLiberals',
    'UnhingedLeft',
    'CHOOSECRUZ',
    'RedTsunamiNov6',
    'whitepower',
    'prolife',
    'TeamDino',
    'waelex',
    'wapol',
    'betteroffnow',
    'votejaime',
    'wapol',
    'gregformontana',
    'teamgiantforte',
    'watergrab',
    'deandelivers',
    'AngryDemocratMob',
    'bluelivesmatter'
]])


In [66]:
infile = 'candidate_handles3B2018-11-02.json'

In [10]:
with open('candidate_handles.txt') as f:
    partisan_tweets_per_candidate = {}
    handles = []
    for line in f:
        partisan_tweets_per_candidate[line[:-1].lower()] = {'liberal': 0, 'conservative': 0, 'neutral': 0}
        handles.append(line[:-1].lower())

In [61]:
blue_pattern = re.compile(".*blue*")
bool(blue_pattern.match('voteblue'))

True

In [146]:
sys.stdout.write("parsing " + infile + "...")
sys.stdout.flush()

with open(infile) as f:
    blue_pattern = re.compile(".*blue*")
    red_pattern = re.compile(".*red*")
    for line in f:
        if line != None:
            tweet = json.loads(line.rstrip('\n'))
            try:
                # Classify tweet as liberal or conservative
                lib_score = 0
                cons_score = 0
                tweet_sentiment = 'neutral'
                for hashtag in tweet['entities']['hashtags']:
                    ht = hashtag['text'].lower()
                    if ht in lib_hashtags or blue_pattern.match(ht):
                        lib_score +=1
                    elif ht in cons_hashtags or red_pattern.match(ht):
                        cons_score += 1
                if lib_score > cons_score:
                    tweet_sentiment = 'liberal'
#                         print(tweet['text'].lower())
                elif lib_score < cons_score:
                    tweet_sentiment = 'conservative'
                else:
                    if tweet['entities']['hashtags']:
                        print(tweet['entities']['hashtags'])

                # Get the candidates mentioned in tweets
                mentions = re.findall('@[a-zA-Z0-9_]{1,15}', tweet['text'].lower())
                # Update dictionary
                for person in mentions:
                    try:
                        partisan_tweets_per_candidate[person][tweet_sentiment] += 1
                    except:
                        pass;
#                     print(tweet_sentiment)
            except:
                print('FAILED')


NameError: name 'infile' is not defined

In [68]:
partisan_tweets_per_candidate

{'@kennedy4alabama': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@repbyrne': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@tabithak': {'conservative': 0, 'liberal': 0, 'neutral': 104},
 '@repmartharoby': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@haganforhouse': {'conservative': 0, 'liberal': 49, 'neutral': 128},
 '@repmikerogersal': {'conservative': 0, 'liberal': 0, 'neutral': 15},
 '@leeauman2018': {'conservative': 0, 'liberal': 0, 'neutral': 3},
 '@robert_aderholt': {'conservative': 0, 'liberal': 0, 'neutral': 44},
 '@peter_joffrion': {'conservative': 0, 'liberal': 0, 'neutral': 72},
 '@repmobrooks': {'conservative': 0, 'liberal': 0, 'neutral': 26},
 '@votedannerkline': {'conservative': 0, 'liberal': 0, 'neutral': 0},
 '@usrepgarypalmer': {'conservative': 0, 'liberal': 0, 'neutral': 12},
 '@repterrisewell': {'conservative': 0, 'liberal': 0, 'neutral': 3},
 '@repdonyoung': {'conservative': 0, 'liberal': 0, 'neutral': 80},
 '@alysegalvin': {'conservative': 0, 'li

## Predicting election outcomes

We have a map from candidate name to Twitter handle, and the csv with district, state, chamber, party affiliation, and incumbency data for each candidate. We want a dataframe with district, state, chamber, projected winner's name, party, incumbency for social influence model, and projected winner's name, party, incumbency for voter count model.

input: dictionary that maps name to handle

In [7]:
candidate_data = pd.read_csv('../candidate_data_final.csv')

In [8]:
candidate_data

,district,name,state,chamber,party,incumbent
0,District 1,Robert Kennedy Jr.,Alabama,house,Democratic Party,False
1,District 1,Bradley Byrne,Alabama,house,Republican Party,True
2,District 2,Tabitha Isner,Alabama,house,Democratic Party,False
3,District 2,Martha Roby,Alabama,house,Republican Party,True
4,District 3,Mallory Hagan,Alabama,house,Democratic Party,False
5,District 3,Mike Rogers,Alabama,house,Alabama Party,True
6,District 4,Lee Auman,Alabama,house,Democratic Party,False
7,District 4,Robert Aderholt,Alabama,house,Republican Party,True
8,District 5,Peter Joffrion,Alabama,house,Democratic Party,False
9,District 5,Mo Brooks,Alabama,house,Republican Party,True


In [9]:
with open('candidate_names_to_handles.pkl', 'rb') as f:
    candidate_names_to_handles = pickle.load(f)

In [10]:
candidate_data

,district,name,state,chamber,party,incumbent
0,District 1,Robert Kennedy Jr.,Alabama,house,Democratic Party,False
1,District 1,Bradley Byrne,Alabama,house,Republican Party,True
2,District 2,Tabitha Isner,Alabama,house,Democratic Party,False
3,District 2,Martha Roby,Alabama,house,Republican Party,True
4,District 3,Mallory Hagan,Alabama,house,Democratic Party,False
5,District 3,Mike Rogers,Alabama,house,Alabama Party,True
6,District 4,Lee Auman,Alabama,house,Democratic Party,False
7,District 4,Robert Aderholt,Alabama,house,Republican Party,True
8,District 5,Peter Joffrion,Alabama,house,Democratic Party,False
9,District 5,Mo Brooks,Alabama,house,Republican Party,True


In [11]:
len(candidate_names_to_handles)

1180

In [12]:
candidate_names_to_handles['Robert Kennedy Jr.']

'Kennedy4Alabama'

In [13]:
def map_names_to_handles(row):
    try:
        return candidate_names_to_handles[row['name']].lower()
    except:
        pass

In [14]:
candidate_data['handles'] = candidate_data.apply(map_names_to_handles, axis=1)

In [15]:
candidate_data[candidate_data['handles'] == None]

,district,name,state,chamber,party,incumbent,handles


In [16]:
len(candidate_data[candidate_data["handles"].isnull()])

61

In [17]:
candidate_data.fillna('hello there')

,district,name,state,chamber,party,incumbent,handles
0,District 1,Robert Kennedy Jr.,Alabama,house,Democratic Party,False,kennedy4alabama
1,District 1,Bradley Byrne,Alabama,house,Republican Party,True,repbyrne
2,District 2,Tabitha Isner,Alabama,house,Democratic Party,False,tabithak
3,District 2,Martha Roby,Alabama,house,Republican Party,True,repmartharoby
4,District 3,Mallory Hagan,Alabama,house,Democratic Party,False,haganforhouse
5,District 3,Mike Rogers,Alabama,house,Alabama Party,True,repmikerogersal
6,District 4,Lee Auman,Alabama,house,Democratic Party,False,leeauman2018
7,District 4,Robert Aderholt,Alabama,house,Republican Party,True,robert_aderholt
8,District 5,Peter Joffrion,Alabama,house,Democratic Party,False,peter_joffrion
9,District 5,Mo Brooks,Alabama,house,Republican Party,True,repmobrooks


In [27]:
races = candidate_data.astype(str).groupby(['district', 'state', 'chamber'])
races

In [19]:
list((5,4))

[5, 4]

In [28]:
len(races)

439

In [21]:
len(races.groups)

439

In [29]:
races.groups

{('District 1', 'Alabama', 'house'): Int64Index([0, 1], dtype='int64'),
 ('District 1', 'Arizona', 'house'): Int64Index([17, 18, 19], dtype='int64'),
 ('District 1', 'Arkansas', 'house'): Int64Index([41, 42, 43], dtype='int64'),
 ('District 1', 'California', 'house'): Int64Index([53, 54], dtype='int64'),
 ('District 1',
  'Colorado',
  'house'): Int64Index([158, 159, 160, 161], dtype='int64'),
 ('District 1',
  'Connecticut',
  'house'): Int64Index([193, 194, 195], dtype='int64'),
 ('District 1', 'Georgia', 'house'): Int64Index([270, 271], dtype='int64'),
 ('District 1',
  'Hawaii',
  'house'): Int64Index([304, 305, 306, 307, 308], dtype='int64'),
 ('District 1',
  'Idaho',
  'house'): Int64Index([311, 312, 313, 314, 315, 316, 317, 318], dtype='int64'),
 ('District 1',
  'Illinois',
  'house'): Int64Index([321, 322, 323], dtype='int64'),
 ('District 1',
  'Indiana',
  'house'): Int64Index([363, 364, 365], dtype='int64'),
 ('District 1', 'Iowa', 'house'): Int64Index([386, 387, 388], dty

In [23]:
for race in races.groups.keys():
    print(race)

('District 1', 'Alabama', 'house')
('District 2', 'Alabama', 'house')
('District 3', 'Alabama', 'house')
('District 4', 'Alabama', 'house')
('District 5', 'Alabama', 'house')
('District 6', 'Alabama', 'house')
('District 7', 'Alabama', 'house')
(nan, 'Alaska', 'house')
('District 1', 'Arizona', 'house')
('District 2', 'Arizona', 'house')
('District 3', 'Arizona', 'house')
('District 4', 'Arizona', 'house')
('District 5', 'Arizona', 'house')
('District 6', 'Arizona', 'house')
('District 7', 'Arizona', 'house')
('District 8', 'Arizona', 'house')
('District 9', 'Arizona', 'house')
('District 1', 'Arkansas', 'house')
('District 2', 'Arkansas', 'house')
('District 3', 'Arkansas', 'house')
('District 4', 'Arkansas', 'house')
('District 1', 'California', 'house')
('District 2', 'California', 'house')
('District 3', 'California', 'house')
('District 4', 'California', 'house')
('District 5', 'California', 'house')
('District 6', 'California', 'house')
('District 7', 'California', 'house')
('Dis

In [37]:
# new_df = pd.DataFrame(columns = ['district','state', 'chamber', 'winner'], index = range(len(races)))
new_df = pd.DataFrame(columns=['district','state', 'chamber', 'winner'])
#rows = []
i = 0
# for i in range(len(races)):
# for race in races.groups.keys():
for race, df in races:
#     race, df = races.groups[i]
#     print(races.groups[race])
    print(df)
#     print(get_winner(df, False, partisan_tweets_per_candidate))

    #winner = get_winner(df, True, partisan_tweets_per_candidate)
    new_df = new_df.append({'district': race[0], 'state': race[1], 'chamber': race[2], 'winner': get_winner(df, True, partisan_tweets_per_candidate)}, ignore_index=True)
#     new_df.loc[i] = pandas.Series({'district': race[0], 'state': race[1], 'chamber': race[2], 'winner': get_winner(df, False, partisan_tweets_per_candidate)})
    i += 1
    #rows.append({'district': race[0], 'state': race[1], 'chamber': race[2], 'winner': get_winner(df, False, partisan_tweets_per_candidate).values[0]})
#new_df = new_df.append(rows)
#     print(df.loc[df['handles'] == handle]['party'].values[0])
#     df.loc['winner'] = get_winner(df, True, partisan_tweets_per_candidate)
#     print(df)

     district                name    state chamber             party  \
0  District 1  Robert Kennedy Jr.  Alabama   house  Democratic Party   
1  District 1       Bradley Byrne  Alabama   house  Republican Party   

  incumbent          handles  
0     False  kennedy4alabama  
1      True         repbyrne  
      district            name    state chamber              party incumbent  \
17  District 1  Tom O'Halleran  Arizona   house   Democratic Party      True   
18  District 1    Wendy Rogers  Arizona   house   Republican Party     False   
19  District 1     David Shock  Arizona   house  Independent Party     False   

          handles  
17   tomohalleran  
18  wendyrogersaz  
19        dbrauer  
      district           name     state chamber              party incumbent  \
41  District 1  Rick Crawford  Arkansas   house   Republican Party      True   
42  District 1  Chintan Desai  Arkansas   house   Democratic Party     False   
43  District 1  Elvis Presley  Arkansas   house  

672      Working Families Party Party     False            None  
       district              name           state chamber             party  \
734  District 1  G.K. Butterfield  North_Carolina   house  Democratic Party   
735  District 1     Roger Allison  North_Carolina   house  Republican Party   

    incumbent        handles  
734      True  gkbutterfield  
735     False  rogerwallison  
       district           name state chamber                party incumbent  \
770  District 1   Steve Chabot  Ohio   house     Republican Party      True   
771  District 1  Aftab Pureval  Ohio   house     Democratic Party     False   
772  District 1    Dirk Kubala  Ohio   house    Libertarian Party     False   
773  District 1  Kiumars Kiani  Ohio   house  No Party Preference     False   

            handles  
770  repstevechabot  
771    aftabpureval  
772      dirkkubala  
773       drkiumars  
       district        name     state chamber             party incumbent  \
814  District 1  Tim

        district              name    state chamber             party  \
292  District 11  Barry Loudermilk  Georgia   house  Republican Party   
293  District 11  Flynn Broady Jr.  Georgia   house  Democratic Party   
294  District 11     Bar-Kim Green  Georgia   house       Independent   

    incumbent         handles  
292     False   reploudermilk  
293     False  flynndbroadyjr  
294     False            None  
        district         name     state chamber             party incumbent  \
344  District 11  Bill Foster  Illinois   house  Democratic Party      True   
345  District 11  Nick Stella  Illinois   house  Republican Party     False   

           handles  
344  repbillfoster  
345   nickstellamd  
        district              name     state chamber              party  \
535  District 11     Haley Stevens  Michigan   house   Democratic Party   
536  District 11      Lena Epstein  Michigan   house   Republican Party   
537  District 11  Leonard Schwartz  Michigan   house 

        district            name  state chamber              party incumbent  \
951  District 13  Mac Thornberry  Texas   house   Republican Party      True   
952  District 13      Greg Sagan  Texas   house   Democratic Party     False   
953  District 13  Calvin DeWeese  Texas   house  Libertarian Party     False   

           handles  
951     mactxpress  
952        gtsagan  
953  calvindeweese  
       district             name       state chamber             party  \
79  District 14    Jackie Speier  California   house  Democratic Party   
80  District 14  Cristina Osmeña  California   house  Republican Party   

   incumbent         handles  
79      True       repspeier  
80     False  cristinaosmena  
        district          name    state chamber             party incumbent  \
243  District 14  Kathy Castor  Florida   house  Democratic Party      True   

          handles  
243  usrepkcastor  
        district           name    state chamber             party incumbent  \


        district              name     state chamber               party  \
707  District 17        Nita Lowey  New_York   house    Democratic Party   
708  District 17  Joseph Ciardullo  New_York   house  Reform Party Party   

    incumbent       handles  
707      True     nitalowey  
708     False  joeciardullo  
        district           name         state chamber             party  \
880  District 17     Conor Lamb  Pennsylvania   house  Democratic Party   
881  District 17  Keith Rothfus  Pennsylvania   house  Republican Party   

    incumbent       handles  
880      True   conorlambpa  
881      True  keithrothfus  
        district             name  state chamber              party incumbent  \
964  District 17      Bill Flores  Texas   house   Republican Party      True   
965  District 17     Rick Kennedy  Texas   house   Democratic Party     False   
966  District 17  Peter Churchman  Texas   house  Libertarian Party     False   

             handles  
964    repbillflo

...

153     False  mando_garza_jr  
        district          name       state chamber             party  \
154  District 52  Scott Peters  California   house  Democratic Party   
155  District 52   Omar Qudrat  California   house  Republican Party   

    incumbent        handles  
154      True  scottpeterssd  
155     False    omarqudrat1  
        district             name       state chamber             party  \
156  District 53      Susan Davis  California   house  Democratic Party   
157  District 53  Morgan Murtaugh  California   house  Republican Party   

    incumbent         handles  
156      True   repsusandavis  
157     False  morganmurtaugh  
      district          name    state chamber             party incumbent  \
10  District 6  Danner Kline  Alabama   house  Democratic Party     False   
11  District 6   Gary Palmer  Alabama   house  Republican Party      True   

            handles  
10  votedannerkline  
11  usrepgarypalmer  
      district              name    st

192     False  coachhaughey_dv  
       district              name    state chamber             party  \
229  District 7  Stephanie Murphy  Florida   house  Democratic Party   
230  District 7       Mike Miller  Florida   house  Republican Party   

    incumbent          handles  
229      True  smurphycongress  
230     False   mike_miller_fl  
       district               name    state chamber             party  \
282  District 7        Rob Woodall  Georgia   house  Republican Party   
283  District 7  Carolyn Bourdeaux  Georgia   house  Democratic Party   

    incumbent        handles  
282     False  reprobwoodall  
283     False    carolyn4ga7  
       district            name     state chamber             party incumbent  \
336  District 7  Danny K. Davis  Illinois   house  Democratic Party      True   
337  District 7   Craig Cameron  Illinois   house  Republican Party     False   

            handles  
336   repdannydavis  
337  craigcameron07  
       district          nam

       district             name state chamber             party incumbent  \
790  District 8  Warren Davidson  Ohio   house  Republican Party      True   
791  District 8    Vanessa Enoch  Ohio   house  Democratic Party     False   

            handles  
790  warrendavidson  
791        drvenoch  
       district             name         state chamber             party  \
861  District 8  Matt Cartwright  Pennsylvania   house  Democratic Party   
862  District 8       John Chrin  Pennsylvania   house  Republican Party   

    incumbent        handles  
861     False  repcartwright  
862     False      johnchrin  
       district          name  state chamber              party incumbent  \
935  District 8   Kevin Brady  Texas   house   Republican Party      True   
936  District 8  Steven David  Texas   house   Democratic Party     False   
937  District 8  Chris Duncan  Texas   house  Libertarian Party     False   

           handles  
935  repkevinbrady  
936     stevedavid  
937  

     district             name  state chamber              party incumbent  \
1110      nan       Angus King  Maine  senate  Independent Party      True   
1111      nan      Eric Brakey  Maine  senate   Republican Party     False   
1112      nan  Zak Ringelstein  Maine  senate   Democratic Party     False   

            handles  
1110   senangusking  
1111  senatorbrakey  
1112           None  
     district           name     state chamber             party incumbent  \
1113      nan     Ben Cardin  Maryland  senate  Democratic Party      True   
1114      nan  Tony Campbell  Maryland  senate  Republican Party     False   

             handles  
1113  bencardinformd  
1114     campbell4md  
     district              name          state chamber             party  \
1134      nan  Elizabeth Warren  Massachusetts  senate  Democratic Party   
1135      nan       Geoff Diehl  Massachusetts  senate  Republican Party   

     incumbent    handles  
1134      True  senwarren  
1135     F

     district           name      state chamber             party incumbent  \
1154      nan  Tammy Baldwin  Wisconsin  senate  Democratic Party      True   
1155      nan    Leah Vukmir  Wisconsin  senate  Republican Party     False   

           handles  
1154  tammybaldwin  
1155    leahvukmir  
     district                   name    state chamber  \
1096      nan             Liz Cheney  Wyoming   house   
1097      nan            Greg Hunter  Wyoming   house   
1098      nan  Daniel Clyde Cummings  Wyoming   house   
1099      nan       Richard Brubaker  Wyoming   house   

                                 party incumbent       handles  
1096  Green check mark transparent.png     False    liz_cheney  
1097                  Democratic Party     False   usawatchdog  
1098          Constitution Party Party     False          None  
1099                 Libertarian Party     False  richbrubaker  
     district           name    state chamber             party incumbent  \
1130      n

In [38]:
new_df

,district,state,chamber,winner
0,District 1,Alabama,house,Robert Kennedy Jr.
1,District 1,Arizona,house,Wendy Rogers
2,District 1,Arkansas,house,Chintan Desai
3,District 1,California,house,Audrey Denney
4,District 1,Colorado,house,Casper Stockham
5,District 1,Connecticut,house,John Larson
6,District 1,Georgia,house,Lisa Ring
7,District 1,Hawaii,house,Cam Cavasso
8,District 1,Idaho,house,Russ Fulcher
9,District 1,Illinois,house,Bobby Rush


In [39]:
new_df.to_csv('total_predictions.csv')

In [32]:
i

439

In [34]:
new_df[new_df['chamber'] == 'house'].shape

(406, 4)

In [240]:
rows

[{'chamber': 'house',
  'district': 'District 1',
  'state': 'Alabama',
  'winner': 'Robert Kennedy Jr.'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Arizona',
  'winner': 'Wendy Rogers'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Arkansas',
  'winner': 'Chintan Desai'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'California',
  'winner': 'Audrey Denney'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Colorado',
  'winner': 'Casper Stockham'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Connecticut',
  'winner': 'John Larson'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Georgia',
  'winner': 'Lisa Ring'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Hawaii',
  'winner': 'Cam Cavasso'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Idaho',
  'winner': 'Cristina McNeil'},
 {'chamber': 'house',
  'district': 'District 1',
  'state': 'Illinois',
  '

In [278]:
new_df

,district,state,chamber,winner
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [245]:
len(new_df[new_df["state"].isnull()])

43

In [26]:
with open('partisan_tweets_per_candidate_final.pkl', 'rb') as f:
    partisan_tweets_per_candidate = pickle.load(f)

In [25]:
def get_winner(df, total, partisan_tweets_per_candidate):
    metric = 'total' if total else 'unique'
    # get data of top two candidates in number of total mentions
    num_of_neutral_tweets, score = dict(), dict()
    
    for handle in df['handles']:
        try:
            num_of_neutral_tweets[handle] = partisan_tweets_per_candidate[handle]['neutral'][metric]
        except:
            pass
    top_two_handles = sorted(num_of_neutral_tweets, key=num_of_neutral_tweets.get, reverse=True)
    affiliation_to_sentiment = {
        'Democratic Party': 'liberal',
        'Republican Party': 'conservative',
        'Libertarian Party': 'conservative',
        'Green Party': 'liberal'
    }
    # we want to add the number of 
    for i in range(len(top_two_handles)):
        handle = top_two_handles[i]
        affiliation = df.loc[df['handles'] == handle]['party'].values[0]
#         print(affiliation)
        try: 
            part1 = partisan_tweets_per_candidate[handle][affiliation_to_sentiment[affiliation]][metric]
            score[handle] = part1 + \
                partisan_tweets_per_candidate[handle][affiliation_to_sentiment[affiliation]][metric]
        except:
            pass
    if score:
        return df.loc[df['handles'] == max(score, key=score.get)]['name'].values[0]
    else:
        return None